# Quickstart
일단 실행해보자!


## Working with data
파이토치는 자체적으로 데이터셋을 제공한다. 이 예제에서는 영상처리 도메인 데이터셋인 TorchVision의 FashionMNIST를 활용한다.

In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

# Download training data from open datasets.
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets.
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

100.0%


Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



100.6%


Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



100.0%


Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



119.3%

Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



데이터셋은 데이터로더에 장착하여 신경망에 연결한다. Batching, sampling, shuffling, multiprocess data loading과 같은 다양한 작업을 자동화한다.

In [3]:
# Create data loaders.
batch_size = 64
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


## Creating Models
딥러닝 모델을 만드는 방법은 다음과 같다.

1. `nn.Module`을 상속하는 클래스를 생성한다.
2. `__init__()`에서 레이어를 초기화한다.
3. `forward()`에서 순전파 연산을 정의한다.

연산가속을 위해 가능하다면 모델을 GPU로 올린다.

In [4]:
# Get cpu or gpu device for training.
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

# Define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

Using cuda device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


## Optimizing the Model Parameters
모델 학습과정은 다음과 같다.

1. Loss function과 Optimizer를 생성한다.
2. 학습과정 `train()`에서,
    1. 데이터셋에 대한 예측을 수행한다.
    2. 예측으로부터 손실을 계산한다.
    3. 손실을 역전파하여 모델을 조정한다.

평가과정 `test()`에서는 예측 후 손실 계산까지만 수행한다.

In [5]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
            
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

이제 학습을 수 에포크 반복하면 된다. 학습 중 Accuracy와 Loss를 출력하여 모델의 예측성능이 향상되고 있는지 확인할 수 있다.

In [6]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.304464  [    0/60000]
loss: 2.292529  [ 6400/60000]
loss: 2.278630  [12800/60000]
loss: 2.269533  [19200/60000]
loss: 2.241519  [25600/60000]
loss: 2.222247  [32000/60000]
loss: 2.224960  [38400/60000]
loss: 2.189769  [44800/60000]
loss: 2.193715  [51200/60000]
loss: 2.157733  [57600/60000]
Test Error: 
 Accuracy: 48.1%, Avg loss: 2.152417 

Epoch 2
-------------------------------
loss: 2.166710  [    0/60000]
loss: 2.159869  [ 6400/60000]
loss: 2.101038  [12800/60000]
loss: 2.113439  [19200/60000]
loss: 2.061358  [25600/60000]
loss: 2.002766  [32000/60000]
loss: 2.030828  [38400/60000]
loss: 1.945781  [44800/60000]
loss: 1.958823  [51200/60000]
loss: 1.887726  [57600/60000]
Test Error: 
 Accuracy: 54.5%, Avg loss: 1.881798 

Epoch 3
-------------------------------
loss: 1.917236  [    0/60000]
loss: 1.894086  [ 6400/60000]
loss: 1.769396  [12800/60000]
loss: 1.809460  [19200/60000]
loss: 1.703539  [25600/60000]
loss: 1.651712  [32000/600

## Saving & Loading Models
- 모델을 저장하는 일반적인 방법은 inertial state dictionary를 직렬화하는 것이다.
- 모델을 불러올 때에는 새로운 모델을 생성한 뒤 state dictionary를 붙여넣는다.

In [7]:
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

model = NeuralNetwork()
model.load_state_dict(torch.load("model.pth"))

Saved PyTorch Model State to model.pth


<All keys matched successfully>

불러온 모델을 사용해보자.

In [8]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Ankle boot", Actual: "Ankle boot"
